In [3]:
import os
import pandas as pd
import openai
import pinecone
from tqdm.notebook import tqdm

In [4]:
"""
    Generates a high-dimensional embedding for the given text using OpenAI's embedding model.

    Parameters:
        text: Input text to embed.
        model: Embedding model name (default is "text-embedding-3-small").

    Returns:
        A list representing the embedding vector.
 """
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )
    embedding = response.data[0].embedding
    return embedding

In [6]:
pc = pinecone.Pinecone(api_key=pinecone_api_key)
index = pc.Index("gen-fit")
messages = []

In [5]:
# Example query text
query_text = "Want to know about replacements for sugar"

# Generate embedding for the query
query_embedding = get_embedding(query_text)

In [7]:
# Example query text
# query_text = "Want to know about replacements for sugar"

# Generate embedding for the query
# query_embedding = get_embedding(query_text)

"""
    Retrieves the top 5 matching documents from the Pinecone index using the query embedding.

    Parameters:
        query_embedding: The embedding vector of the user query.

    Returns:
        A dictionary containing the top matching documents and their metadata.
"""
def retrieve_documents(query_embedding):

    # Query Pinecone index
    results = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
    )

    # Display the results
    for match in results['matches']:
        print(f"Score: {match['score']}")
        print(f"Title: {match['metadata']['Topic']}")
        print(f"Content: {match['metadata']['Content']}\n")
    
    return results

In [8]:
"""
    Generates a prompt for the GPT model using the top retrieved documents and the original query.

    Parameters:
        documents: The retrieved documents from Pinecone, including metadata.
        original_query: The user's original query.

    Returns:
        A formatted string prompt containing the query and summarized document content.
"""


def generate_prompt(documents, original_query):
    document_texts = []
    for i, match in enumerate(documents['matches'][:3]):
        doc_text = match['metadata'].get('Content', '')
        document_texts.append(f"Document {i+1}: {doc_text}")

    prompt = (
        f"The user's query is: '{original_query}'\n\n"
        "Based on the following documents, answer the users query" +
        "\n".join(document_texts) +
        f"\n\nPlease provide a response in a conversational format. Give response under 300 tokens" +
        "Give the response as a small paragrah"
    )
    return prompt

In [9]:
"""
    Generates a conversational response using GPT-4 based on the provided prompt.

    Parameters:
        prompt: The formatted string prompt containing the query and document context.

    Returns:
        The generated response as a string.
"""


def get_response(prompt):
    messages.append({"role": "user", "content": prompt})
    response = openai.chat.completions.create(
        model='gpt-4', 
        messages=messages,
        max_tokens=1200,
        n=1,
        temperature=0.7,
    )
    questions = response.choices[0].message.content
    messages.append({"role": "assistant", "content": questions})
    return questions

In [ ]:
"""
    Handles the entire query processing pipeline by generating an embedding for the query, 
    retrieving relevant documents, creating a prompt, and generating a conversational response.
    
    Additionally, calculates relevance score and response time for evaluation, with detailed timing 
    for individual steps.

    Parameters:
        query: The user's input query as a string.
"""

import time
from sklearn.metrics.pairwise import cosine_similarity

def process_query(query):
   
    embedding_start_time = time.time()
    query_embedding = get_embedding(query)
    embedding_end_time = time.time()

    if query_embedding is None:
        print("Failed to generate embedding for the query.")
        return
    
    # Calculate time for Query Embedding Generation
    embedding_time = embedding_end_time - embedding_start_time

    retrieval_start_time = time.time()
    results = retrieve_documents(query_embedding)
    retrieval_end_time = time.time()

    if not results['matches']:
        print("No documents were retrieved.")
        return
    
    # Calculate time for Document Retrieval
    retrieval_time = retrieval_end_time - retrieval_start_time

    response_start_time = time.time()
    prompt = generate_prompt(results, query)
    get_response_ans = get_response(prompt)
    response_end_time = time.time()

    # Calculate time for Response Generation
    response_time = response_end_time - response_start_time

    # Calculate relevance score
    response_embedding = get_embedding(get_response_ans)
    relevance_score = cosine_similarity([query_embedding], [response_embedding])[0][0]

    # Total latency calculation
    total_latency = embedding_time + retrieval_time + response_time

    print("\n=== User's Query ===")
    print(query)
    print(f"\n=== Generated Response ===")
    print(get_response_ans)
    print(f"\n=== Metrics ===")
    print(f"Relevance Score: {relevance_score:.2f}")
    print(f"Query Embedding Generation Time: {embedding_time:.2f} seconds")
    print(f"Document Retrieval Time: {retrieval_time:.2f} seconds")
    print(f"Response Generation Time: {response_time:.2f} seconds")
    print(f"Total Response Time (Latency): {total_latency:.2f} seconds")


In [ ]:
query = input("Enter your query: ")
process_query(query)

Score: 0.475393802
Title: Get them active
Content: exercis keep us healthi less like get sick children realli activ hour day activ doesnt mean play sport go gym could play playground go walk necessarili better child seriou athlet exercis sever hour day make sure exercis isnt eat sleep caus burnout could caus problem immun system

Score: 0.429633439
Title: Sample calisthenics workout
Content: tri exercis part calisthen routin train strength aim eight 12 repetit rep cannot rep first gradual increas rep improv beginn start one set rep work three set develop strength rest 30 90 second set start positionstand straight feet togeth movementlift right knee high comfort touch hand knee lower leg repeat left knee one rep start positionstand feet shoulderwidth apart arm side movementslowli bend hip knee lower buttock eight inch your sit back chair let arm swing forward help balanc keep spine neutral hold slowli return start posit start positionkneel four hand shoulderwidth apart walk hand forward

In [ ]:
query = input("Enter your query: ")
process_query(query)


Score: 0.473706484
Title: What's the healthiest diet?
Content: singl diet nutritionist deem “the healthiest” howev sever style eat expert either design optim health observ healthi consum tradit differ peopl around world style eat tend thing common—they tend plantbas diet emphas healthi fat simpl sugar low sodium favor natur food highli process fare typic much western diet exampleth mediterranean style dietget name food avail variou cultur locat around mediterranean sea heavili emphas minim process fruit veget legum nut whole grain contain moder amount yogurt chees poultri fish oliv oil primari cook fat red meat food ad sugar eaten sparingli besid effect weight loss method eat mediterranean style diet link lower risk heart diseas diabet depress form cancer expert developedth dash dietdietari approach stop hypertens specif hearthealthi regimen combin food type contain diet seem work togeth especi effect lower blood pressur decreas risk heart failur key featur dash low cholesterol satur f

In [ ]:
# In case of interactive session

def generate_prompt(results, query, context=None):
    """
    Generates a prompt for GPT-4 based on retrieved documents and optional past context.

    Parameters:
        results: Retrieved documents from Pinecone.
        query: The user's current input query.
        context: Optional context containing past queries and responses.
    
    Returns:
        A string prompt to send to GPT-4.
    """
    document_texts = []
    for i, match in enumerate(results['matches'][:3]):  # Use top 3 documents
        doc_text = match['metadata'].get('Content', '')
        document_texts.append(f"Document {i+1}: {doc_text}")

    # Add past context if available
    context_text = ""
    if context and 'queries' in context and 'responses' in context:
        for i, (q, r) in enumerate(zip(context['queries'], context['responses'])):
            context_text += f"Previous Query {i+1}: {q}\nPrevious Response {i+1}: {r}\n"

    # Combine context and current query
    prompt = (
        f"The user's query is: '{query}'\n\n"
        "Based on the following documents and past context, answer the user's query:\n\n" +
        context_text + "\n".join(document_texts) +
        "\n\nPlease provide a response in a conversational format, limited to 300 tokens."
    )
    return prompt

"""
    Conversation loop for iterative queries
"""
def interactive_session():
    context = {'queries': [], 'responses': []}
    print("Welcome to the conversational query assistant. Type 'exit' to end the session.\n")
    
    while True:
        query = input("Enter your query: ")
        if query.lower() == 'exit':
            print("Exiting session. Goodbye!")
            break
        context = process_query(query, context)